**TESTING BASIC CONVERSATIONAL AGENT LOGIC**

Goal here is just to see if a conv agent can stick to its role, and if it can follow directions properly

Conclusion atm: agent seems wacky with gpt3.5 and doesn't always follow instrcutions. Maybe gpt4 would follow it. Idea: try with plan-and-execute model.

!! change the API keys once we get the uni ones !!

In [1]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-proj-1xJeptjFfqBedSaJltruT3BlbkFJCGvIrjfkmvFSzxVpRyGD'

os.environ["LANGSMITH_API_KEY"] = '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'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Tutorial"

In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [3]:
# Import additional required modules
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI

import functools
from langchain_core.messages import AIMessage

# Create a conversational agent function
def create_conversational_agent(llm, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            ( #! too many directives might be complicated for the agent bc it keeps performing randomly
                "system",
                "you are a conversational agent. "
                "you are speaking to a student. "
                "Speak in english with the student."
                "as long as the user is making mistakes, they did not learn the lesson properly."
                "You have three missions: teach, evaluate, and converse. "
                "state those missions clearly to the user."
                "1. Teach a lesson based on the user's request. "
                "2. Once you have given your lesson, you must evaluate the user by questioning them, "
                "keep asking the questions until the student gets the answer correct."
                "3. After evaluation, you must have a conversation in the context of the lesson"
                "with the user by asking them questions and talking with them by role playing a scenario,"
                "keep this conversation going with the user until all the lesson material is covered."
                "make sure only the lesson content is needed for the conversation."
                "If the user has successfully learned the lesson and you did all three missions"
                "Provide a summary of what the user has learned and then quit and stop the conversation. "
                "write clearly: lesson learned. "
                "{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    return prompt | llm


In [4]:

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # Convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Track the sender to know who to pass to next
        "sender": name,
    }


# Initialize the language model
llm = ChatOpenAI(model="gpt-4o")

# Create the conversational agent
conversation_agent = create_conversational_agent(
    llm,
    system_message="You are a teacher for the English language.",
)
teacher_node = functools.partial(agent_node, agent=conversation_agent, name="convo-bot")

c:\Users\titouan\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
# graph_builder.add_node("conversation_bot", teacher_node)

# graph_builder.set_entry_point("convo-bot")
# graph_builder.set_finish_point("convo-bot")
# graph = graph_builder.compile()


In [6]:
# Function to start the conversation
def start_conversation():
    messages = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Conversation ended.")
            break

        messages.append(HumanMessage(content=user_input))
        state = {"messages": messages}
        
        result = teacher_node(state)
        messages.append(result["messages"][0])
        
        print(f"User: {user_input}")
        print(f"Agent: {result['messages'][0].content}")

        # Check if the agent decides to stop the conversation
        if "lesson learned" in result['messages'][0].content.lower():
            print("Agent has ended the conversation.")
            break

In [7]:
start_conversation()

User: hello i am A1 luxembourgish student please teach me greetings
Agent: Hello! I'd be happy to help you with greetings in English. Here are my three missions:

1. Teach a lesson based on your request.
2. Evaluate you by asking questions until you get the answers correct.
3. Have a conversation with you in the context of the lesson to ensure you've learned it.

Let's start with some basic English greetings:

1. **Hello** - A common greeting used at any time of the day.
2. **Hi** - A more casual way to say hello.
3. **Good morning** - Used in the morning until around noon.
4. **Good afternoon** - Used after noon until about 6 PM.
5. **Good evening** - Used after 6 PM until bedtime.
6. **How are you?** - A polite way to ask about someone's well-being.
7. **Nice to meet you** - Used when you meet someone for the first time.

Now, I'll evaluate you by asking some questions. Please answer them:

1. What would you say if you meet someone in the morning?
2. How would you greet someone in th